In [ ]:
# default_exp lyufunc

# Functions
> all self-defined functions can be used to extract lines from LiDAR data.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from lyutool.lyuds import *
import numpy as np

In [ ]:
#export
def generateLPByIDS(idx):
    """generate a Lpoints class based on a point list.
    
       idx : bool type like [ True False  True False  True False  True  True False False]
    """
    a = np.argwhere(idx == True)
    a = a.T
    nid = a.ravel()
    return LPoints(nid)

In [ ]:
np.random.seed(1)
a = np.random.randint(0,80,10)
print(a)
idx = a > 30
print(idx)
lp = generateLPByIDS(idx)
print([i for i in lp.dic.keys()])
print([i for i in lp.dic.values()])
print(np.array(list(lp.dic.values())))
a[list(lp.dic.values())]

[37 12 72  9 75  5 79 64 16  1]
[ True False  True False  True False  True  True False False]
[0, 1, 2, 3, 4]
[0, 2, 4, 6, 7]
[0 2 4 6 7]


array([37, 72, 75, 79, 64])

In [ ]:
#export
def getPointsFromSource(**kwargs):
    """ Based on the class LPoints, obtain points from the original data.
    
        {'LP': lp, 'POINTS':xyz}
    """
    lp = kwargs['LP']
    xyz = kwargs['POINTS']
    return xyz[list(lp.dic.values())]

In [ ]:
np.random.seed(1)
a = np.random.randint(0,80,10)
print(a)
idx = a > 30
print(idx)
lp = generateLPByIDS(idx)
kwargs = {'LP': lp, 'POINTS':a}
getPointsFromSource(**kwargs)

[37 12 72  9 75  5 79 64 16  1]
[ True False  True False  True False  True  True False False]


array([37, 72, 75, 79, 64])

In [ ]:
np.random.seed(1)
a = np.random.randint(0,80,(8,3))
print(a)
idx = a[:,2] > 30
print(idx)
lp = generateLPByIDS(idx)
kwargs = {'LP': lp, 'POINTS':a}
getPointsFromSource(**kwargs)

[[37 12 72]
 [ 9 75  5]
 [79 64 16]
 [ 1 76 71]
 [ 6 25 50]
 [20 18 11]
 [28 29 14]
 [50 68 13]]
[ True False False  True  True False False False]


array([[37, 12, 72],
       [ 1, 76, 71],
       [ 6, 25, 50]])

In [ ]:
#export
def locatePointsFromBuffer_3D(**kwargs):
    """
    Description: obtain points from original point cloud base on the image buffer in 3D space
    
                self-defined the range of Z axis (Height) is used.
    
    """
    img_buffer = kwargs['img_buffer']
    min_xyz = kwargs['min_xyz']
    pts = kwargs['pts']
    cellsize = kwargs['cellsize']
    zrange = kwargs['zrange'] # [z_min, z_max]
    z_min, z_max = zrange
    #
    ly, lx = np.nonzero(img_buffer)
    x_min, y_min = min_xyz[0:2]
    px = cellsize * lx + x_min
    py = cellsize * ly + y_min
    nid_line=[]
    zipped = zip(px,py)
    for i,j in zipped:
        fcx = np.logical_and(pts[:,0] >= i, pts[:,0] < (i + cellsize))
        fcy = np.logical_and(pts[:,1] >= j, pts[:,1] < (j + cellsize))
        fcz = np.logical_and(pts[:,2] >= z_min, pts[:,2] <= z_max)
        idx = np.logical_and(fcx, fcy)
        idx = np.logical_and(idx, fcz)
        a = np.argwhere(idx==True)
        a = a.ravel()
        nid_line = nid_line + a.tolist()
    return nid_line

In [ ]:
#export
def calOutlierByIQR(a):
    """
    Calculating the Outlier Fences Using the Interquartile Range.
    return: [lower inner fence, lower outer fence, upper inner fence, upper outer fence]
    """
    iqr = np.quantile(a,0.75) - np.quantile(a,0.25)
    inner_para = iqr * 1.5
    outer_para = iqr * 3
    lower_inner_fence = np.quantile(a,0.25) - inner_para
    lower_outer_fence = np.quantile(a,0.25) - outer_para
    upper_inner_fence = np.quantile(a,0.75) + inner_para
    upper_outer_fence = np.quantile(a,0.75) + outer_para
    return {'lower_inner_fence': lower_inner_fence,
            'lower_outer_fence': lower_outer_fence,
            'upper_inner_fence': upper_inner_fence,
            'upper_outer_fence': upper_outer_fence
           }

In [ ]:
a = [0, 50,52, 55, 54, 58, 100, 101,1]
d = calOutlierByIQR(a)
d

{'lower_inner_fence': 38.0,
 'lower_outer_fence': 26.0,
 'upper_inner_fence': 70.0,
 'upper_outer_fence': 82.0}

In [ ]:
#export
def generateLineBordersByBuffer(**kwargs):
    """
    generate borders of powerlines based on the corridors
    """
    img_buffer = kwargs['img_buffer']
    min_xyz = kwargs['min_xyz']
    pts = kwargs['pts'] # deep copy of the original data
    #
    b = np.zeros((pts.shape[0],2))
    pts = np.hstack((pts,b))
    #
    cellsize = kwargs['cellsize']
    ly, lx = np.nonzero(img_buffer)
    x_min, y_min = min_xyz[0:2]
    px = cellsize * lx + x_min
    py = cellsize * ly + y_min
    nid_line=[]
    zipped = zip(px,py)
    for i,j in zipped:
        fcx = np.logical_and(pts[:,0] >= i, pts[:,0] < (i + cellsize))
        fcy = np.logical_and(pts[:,1] >= j, pts[:,1] < (j + cellsize))
        idx = np.logical_and(fcx, fcy)
        a = pts[idx,2]
        if len(a) == 0:
            continue
        dic_border = calOutlierByIQR(a)
        lower_inner_fence = dic_border['lower_inner_fence']
        upper_inner_fence = dic_border['upper_inner_fence']
        pts[idx, 3] = lower_inner_fence
        pts[idx, 4] = upper_inner_fence
        #
        a = np.argwhere(idx==True)
        a = a.ravel()
        nid_line = nid_line + a.tolist()
    return pts[np.unique(nid_line)]

In [ ]:
#export
def extractLineFromTarget(**kwargs):
    """
    based on the corridor and the related points, extract lines from original point clouds.
    """
    img_buffer = kwargs['img_buffer']
    min_xyz = kwargs['min_xyz']
    cellsize = kwargs['cellsize']
    pts_target = kwargs['pts_target'] # deep copy of the original data
    pts_buffer = kwargs['pts_buffer']
    zrange = kwargs['zrange']
    zmin,zmax = zrange
    ly, lx = np.nonzero(img_buffer)
    x_min, y_min = min_xyz[0:2]
    px = cellsize * lx + x_min
    py = cellsize * ly + y_min
    nid_line=[]
    zipped = zip(px,py)
    for i,j in zipped:
        fcx = np.logical_and(pts_buffer[:,0] >= i, pts_buffer[:,0] < (i + cellsize))
        fcy = np.logical_and(pts_buffer[:,1] >= j, pts_buffer[:,1] < (j + cellsize))
        idx = np.logical_and(fcx, fcy)
        a = pts_buffer[idx,2]
        if len(a) == 0:
            continue
        dic_border = calOutlierByIQR(a)
        lower_inner_fence = dic_border['lower_inner_fence']
        upper_inner_fence = dic_border['upper_inner_fence']
        lower_inner_fence = max(lower_inner_fence, min(a))
        upper_inner_fence = min(upper_inner_fence, max(a))
        # begin to filter target point cloud
        fcx = np.logical_and(pts_target[:,0] >= i, pts_target[:,0] < (i + cellsize))
        fcy = np.logical_and(pts_target[:,1] >= j, pts_target[:,1] < (j + cellsize))
        fcz = np.logical_and(pts_target[:,2] >  lower_inner_fence, pts_target[:,2] < upper_inner_fence)
        idx = np.logical_and(fcx, fcy)
        idx = np.logical_and(fcz, idx)
        a = np.argwhere(idx==True)
        a = a.ravel()
        nid_line = nid_line + a.tolist()
    return nid_line

In [ ]:
# export
def extractLineFromTarget_dales(**kwargs):
    """
    based on the corridor and the related points, extract lines from original point clouds.
    """
    img_buffer = kwargs['img_buffer']
    min_xyz = kwargs['min_xyz']
    cellsize = kwargs['cellsize']
    pts_target = kwargs['pts_target'] # deep copy of the original data
    pts_buffer = kwargs['pts_buffer']
    ly, lx = np.nonzero(img_buffer)
    x_min, y_min = min_xyz[0:2]
    px = cellsize * lx + x_min
    py = cellsize * ly + y_min
    nid_line=[]
    zipped = zip(px,py)
    for i,j in zipped:
        fcx = np.logical_and(pts_buffer[:,0] >= i, pts_buffer[:,0] < (i + cellsize))
        fcy = np.logical_and(pts_buffer[:,1] >= j, pts_buffer[:,1] < (j + cellsize))
        idx = np.logical_and(fcx, fcy)
        a = pts_buffer[idx,2]
        if len(a) == 0:
            continue
        dic_border = calOutlierByIQR(a)
        lower_inner_fence = dic_border['lower_inner_fence']
        upper_inner_fence = dic_border['upper_inner_fence']
        #lower_inner_fence = max(lower_inner_fence, min(a))
        #upper_inner_fence = min(upper_inner_fence, max(a))
        # begin to filter target point cloud
        fcx = np.logical_and(pts_target[:,0] >= i, pts_target[:,0] < (i + cellsize))
        fcy = np.logical_and(pts_target[:,1] >= j, pts_target[:,1] < (j + cellsize))
        fcz = np.logical_and(pts_target[:,2] >  lower_inner_fence, pts_target[:,2] < upper_inner_fence)
        idx = np.logical_and(fcx, fcy)
        idx = np.logical_and(fcz, idx)
        a = np.argwhere(idx==True)
        a = a.ravel()
        nid_line = nid_line + a.tolist()
    return nid_line

In [ ]:
# export
def getVoxelKeys(**kwargs):
    pts = kwargs['pts']
    voxelsize = kwargs['voxelsize']
    mask = kwargs['mask']
    pts_min = np.min(pts,axis=0)
    a = (pts[mask] - pts_min) // voxelsize
    a = a.astype(int)
    non_empty_voxel_keys, inverse, nb_pts_per_voxel= np.unique(a, axis=0, return_inverse=True, return_counts=True)
    return non_empty_voxel_keys
# export
def filterPointsByVoxel(**kwargs):
    voxelkey = kwargs['voxelkey']
    voxelsize = kwargs['voxelsize']
    xyz = kwargs['pts']
    if 'con' not in kwargs.keys():
        con = np.ones((xyz.shape[0],),dtype=np.bool)
    else:
        con = kwargs['con']
    
    xyz_min = np.min(xyz,axis=0)
    lidx,lidy,lidz = voxelkey
    uidx,uidy,uidz = lidx+1, lidy+1, lidz+1
    ux = xyz_min[0] + uidx*voxelsize
    uy = xyz_min[1] + uidy*voxelsize
    uz = xyz_min[2] + uidz*voxelsize
    lx = xyz_min[0] + lidx*voxelsize
    ly = xyz_min[1] + lidy*voxelsize
    lz = xyz_min[2] + lidz*voxelsize
    con1 = np.logical_and(xyz[:,0]>lx, xyz[:,0]<=ux)
    con2 = np.logical_and(xyz[:,1]>ly, xyz[:,1]<=uy)
    con3 = np.logical_and(xyz[:,2]>lz, xyz[:,2]<=uz)
    a = np.logical_and(con1, con2)
    b = np.logical_and(con3, con)
    c = np.logical_and(a,b)
    return c